### Informalize

1. Select a random sample of the lean and isabelle data.
2. Query gpt-4 using the OpenAI API
3. Save the outputs to the /data/informalize folder

In [1]:
import pandas as pd
from openai import OpenAI
SAMPLE_SIZE = 2
SEED = 11632

In [8]:
import os
if not os.path.isdir("data/informalizations"):
    os.makedirs("data/informalizations")

In [2]:
df_i = pd.read_json(open("data/modified_mma/isabelle_train.jsonl","r"), lines = True).sample(SAMPLE_SIZE, random_state = SEED)
df_l = pd.read_json(open("data/modified_mma/lean_train.jsonl","r"), lines = True).sample(SAMPLE_SIZE,random_state = SEED)

In [3]:
client = OpenAI(
  api_key="sk-3AoIomsIRrEs37k4d5aRQA",
  base_url="https://cmu.litellm.ai"
)

In [4]:
output_name = []
output_statement = []
output_both = []
from tqdm import tqdm 
for index, row in tqdm(df_i.iterrows()):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages = [{"role":"user","content":row["prompt_name"]}],
        max_tokens = 512,
        n=1
    )
    output_name.append(completion.choices[0].message.content)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages = [{"role":"user","content":row["prompt_statement"]}],
        max_tokens = 512,
        n=1
    )

    output_statement.append(completion.choices[0].message.content)

    completion = client.chat.completions.create(
        model ="gpt-4o",
        messages = [{"role":"user","content":row["prompt_both"]}],
        max_tokens = 512,
        n=1
    )
    output_both.append(completion.choices[0].message.content)
    
df_i.insert(0,"output_both",output_both)
df_i.insert(0,"output_statement",output_statement)
df_i.insert(0,"output_name",output_name)

2it [00:08,  4.34s/it]


In [10]:
df_l.to_csv(f"data/informalizations/isabelle_{SAMPLE_SIZE}")

In [ ]:
client = OpenAI(
  api_key="sk-3AoIomsIRrEs37k4d5aRQA",
  base_url="https://cmu.litellm.ai",
)
output_name = []
output_statement = []
output_both = []
for index, row in df_l.iterrows():
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages = [{"role":"user","content":row["prompt_name"]}],
        max_tokens = 512,
        n=1
    )
    output_name.append(completion.choices[0].message.content)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages = [{"role":"user","content":row["prompt_statement"]}],
        max_tokens = 512,
        n=1
    )

    output_statement.append(completion.choices[0].message.content)

    completion = client.chat.completions.create(
        model ="gpt-4o",
        messages = [{"role":"user","content":row["prompt_both"]}],
        max_tokens = 512,
        n=1
    )
    output_both.append(completion.choices[0].message.content)
    if index in [200,400,600,800,1000,1200,1400,1600,1800]:
        pd.DataFrame({"output_both":output_both,"output_statemet":output_statement, "output_name":output_name}).to_json("lean_running")
df_l.insert(0,"output_both",output_both)
df_l.insert(0,"output_statement",output_statement)
df_l.insert(0,"output_name",output_name)

In [9]:
df_l.to_csv(f"data/informalizations/lean_{SAMPLE_SIZE}")